In [ ]:
# Standard Libraries & Configuration
import os, shutil, yaml, six
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Medical Imaging & Feature Extraction
# import SimpleITK as sitk
# import radiomics
# from radiomics import featureextractor
from PIL import Image
import cv2

# Machine Learning Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from itertools import cycle


In [ ]:
# Configuration for file paths

image_dir = '../dataset/images/'
image_prefix = image_dir + 'ISIC_'
label_prefix = '../dataset/HAM10000_segmentations_lesion_tschandl/ISIC_'
output_dir = '../dataset/mask_ring/'

def visualize_lesion_boundaries(image_list, image_dir, segmentation_dir, kernel_size=50):
    """
    Processes a list of images to visualize the original, dilated, and eroded 
    segmentation boundaries.
    """
    for img_path in image_list:
        # Extract Image ID (e.g., 0024315)
        image_id = img_path.split('_')[1].split('.')[0]
        
        # Construct full paths
        full_image_path = os.path.join(image_dir, f"ISIC_{image_id}.jpg")
        label_path = os.path.join(segmentation_dir, f"ISIC_{image_id}_segmentation.png")
        
        # 1. Load Original Image and Mask
        original_img = cv2.imread(full_image_path)
        mask = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        
        if original_img is None or mask is None:
            print(f"Error loading files for ID: {image_id}")
            continue

        # 2. Perform Morphological Operations (Create the Ring)
        # Dilate to capture surrounding skin; Erode to find core
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        img_dilation = cv2.dilate(mask, kernel, iterations=1)
        img_erosion = cv2.erode(mask, kernel, iterations=1)
        
        # Calculate the perilesional 'ring' (optional, but useful for radiomics)
        perilesional_ring = img_dilation - img_erosion

        # 3. Find Contours for Visualization
        # Original border
        orig_contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Dilated border
        dilated_contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Eroded border
        eroded_contours, _ = cv2.findContours(img_erosion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 4. Draw Overlays
        viz_img = original_img.copy()
        # Draw Original Boundary in Red
        cv2.drawContours(viz_img, orig_contours, -1, (0, 0, 255), 2)
        # Draw Dilated and Eroded Boundaries in Cyan/Yellow
        cv2.drawContours(viz_img, dilated_contours, -1, (255, 255, 0), 2)
        cv2.drawContours(viz_img, eroded_contours, -1, (255, 255, 0), 2)

        # 5. Display Result
        plt.figure(figsize=(8, 8))
        plt.imshow(cv2.cvtColor(viz_img, cv2.COLOR_BGR2RGB))
        plt.title(f"Boundary Visualization: {image_id}")
        plt.axis('off')
        plt.show()
        




In [ ]:
def process_lesion_rings(image_list):
    """
    Creates 'Ring' masks by dilating original segmentation masks to capture 
    lesion borders. This helps in identifying high-impact boundary features.
    """
    for img_path in image_list:
        # Extract Image ID (e.g., ISIC_0024310)
        image_id = img_path.split('_')[1].split('.')[0]
        
        # Construct Paths
        original_img_path = f"{image_prefix}{image_id}.jpg"
        label_path = f"{label_prefix}{image_id}_segmentation.png"
        
        # Load & Process Mask
        mask = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        
        # Find original contours (Blue/Red Line)
        contours_orig, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Perform Dilation to create the 'Ring'
        # Using a 25x25 kernel to capture the surrounding skin context
        kernel = np.ones((25, 25), np.uint8)
        dilated_mask = cv2.dilate(mask, kernel, iterations=1)
        contours_dilated, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Overlay on Original Image
        original_img = cv2.imread(img_path)
        img_with_viz = original_img.copy()
        
        # Draw Original Boundary (Red) and Dilated Boundary (Green)
        cv2.drawContours(img_with_viz, contours_orig, -1, (0, 0, 255), 2)  # Red
        cv2.drawContours(img_with_viz, contours_dilated, -1, (0, 255, 0), 2) # Green
        
        # Save the processed 'Policy-Ready' visualization
        save_path = f"{output_dir}{image_id}_mask_ring.jpg"
        plt.imsave(save_path, cv2.cvtColor(img_with_viz, cv2.COLOR_BGR2RGB))


        



In [ ]:
def detect_hollow_core_masks(image_list, segmentation_dir, kernel_size=50):
    """
    Identifies masks where the lesion core is too thin to survive erosion.
    Used for data quality control and identifying 'hollow' segmentations.
    """
    hollow_core_ids = []
    hollow_count = 0
    
    # Define a square kernel for morphological operations
    kernel = np.ones((kernel_size, kernel_size), np.uint8) 

    for img_path in image_list:
        # Extract ID (e.g., ISIC_0024310)
        img_id = img_path.split('_')[1].split('.')[0]
        label_path = f"{segmentation_dir}/ISIC_{img_id}_segmentation.png"
        
        # Load mask in grayscale
        mask = cv2.imread(label_path, 0)
        
        if mask is not None:
            # Erosion removes the outer boundaries of the white (255) pixels
            img_erode = cv2.erode(mask, kernel, iterations=1)
            
            # If no white pixels remain, the lesion has no 'solid core' 
            # relative to the 50x50 kernel size
            if np.sum(img_erode == 255) == 0:
                hollow_count += 1
                hollow_core_ids.append(img_id)

    # Export findings for reproducibility
    np.savetxt('hollow_core_indices.txt', hollow_core_ids, delimiter=',', fmt='%s')
    
    print(f"Total hollow-core masks found: {hollow_count}")
    return hollow_core_ids


# Load the IDs we identified as having "hollow cores"

hollow_core_ids = np.loadtxt('hollow_core_indices.txt', delimiter=',', dtype=str)

# Filter the image list (im_n) to remove these problematic IDs
# We only want to extract features from high-quality, solid segmentations
filtered_im_n = [path for path in im_n if path.split('_')[1].split('.')[0] not in hollow_core_ids]

print(f"Original images: {len(im_n)}")
print(f"Images after hollow-core removal: {len(filtered_im_n)}")



In [ ]:
# Configuration & Path Setup
PARAMS_PATH = os.path.join(os.getcwd(), 'Params.yaml')
OUTPUT_PATH = '../dataset/PyRadiomics_files/radiomics_features_all.csv'

with open(PARAMS_PATH, 'r') as file:
    params = yaml.safe_load(file)

# Initialize Extractor (Done once outside the loop for efficiency)
extractor = featureextractor.RadiomicsFeatureExtractor(params)
extractor.disableAllFeatures()
extractor.enableAllFeatures()

features_list = []

print(f"Starting extraction for {len(filtered_im_n)} images...")

# Optimized Feature Extraction Loop
for i, img_path in enumerate(filtered_im_n):
    try:
        # Construct file paths
        base_name = img_path.split('.')[0]
        mask_path = f"{base_name}_mask.jpg"
        image_path = f"{base_name}_or.jpg"
        
        # Load images
        mask_sitk = sitk.ReadImage(mask_path)
        image_sitk = sitk.ReadImage(image_path)
        
        # Format: Cast to single channel (Grayscale)
        # Skin images are RGB; Radiomics requires a 2D scalar image
        image_grayscale = sitk.VectorIndexSelectionCast(image_sitk, 0)
        
        # Execute extraction
        # result is a Python OrderedDict
        result = extractor.execute(image_grayscale, mask_sitk)
        
        # Add metadata to the result for identification later
        result['image_id'] = img_path
        features_list.append(result)
        
        if i % 10 == 0:
            print(f"Processed {i}/{len(filtered_im_n)} images...")

    except Exception as e:
        print(f"Error processing {img_path}: {e}")

# Final Data Processing & Saving
# Convert list of OrderedDicts to a pandas DataFrame
df_features = pd.DataFrame(features_list)

# Filter columns: Keep 'image_id' and all 'original_' radiomics features
# This removes diagnostic/header information that Radiomics automatically generates
cols_to_keep = ['image_id'] + [c for c in df_features.columns if c.startswith('original_')]
df_final = df_features[cols_to_keep]

# Save to CSV
df_final.to_csv(OUTPUT_PATH, index=False)
print(f"Extraction complete. Results saved to: {OUTPUT_PATH}")

In [ ]:
# --- Load Extracted Features ---
# (Assuming these CSVs are generated by your extraction loop)
combo_ring = pd.read_csv('../dataset/PyRadiomics_files/radiomics_combo_ring_all.csv')
lesion = pd.read_csv('../dataset/PyRadiomics_files/radiomics_lesion_all.csv')

# --- Merge Features ---
# specific 'inner' merge ensures we only keep images that have BOTH feature sets
radiomics_features = pd.merge(lesion, combo_ring, on='image_id')

# --- Remove Hollow Core Outliers ---
# Load the hollow IDs we found earlier
try:
    hollow_core_ids = np.loadtxt('hollow_core_indices.txt', delimiter=',', dtype=str)
    # Filter them out of the feature set
    # Note: Adjust logic if image_id format differs in CSV
    radiomics_features = radiomics_features[~radiomics_features['image_id'].str.contains('|'.join(hollow_core_ids))]
except OSError:
    print("Warning: hollow_core_indices.txt not found. Skipping hollow core removal.")

In [ ]:
# Comprehensive check for missing values
null_summary = radiomics_features.isnull().sum()

# Filter to display only features with missing data
missing_features = null_summary[null_summary > 0]

if not missing_features.empty:
    print("Missing values detected in the following features:")
    print(missing_features)
    
else:
    print("Data Integrity Check Passed: No missing values found.")

In [ ]:
# Find features with near-zero variance
low_variance = radiomics_features.std()[radiomics_features.std() < 0.01]
print(f"Features to consider dropping due to low variance:\n{low_variance}")

In [ ]:
# 1. Identify low variance features (Standard Deviation < 0.01)
# Note: Ensure you only select numeric columns to avoid errors with string IDs
numeric_features = radiomics_features.select_dtypes(include=[np.number])
low_variance = numeric_features.std()[numeric_features.std() < 0.01]

print(f"Dropping {len(low_variance)} features with low variance.")

# 2. Drop these features from the main dataframe
# We use 'errors=ignore' just in case a feature was already dropped
radiomics_features.drop(columns=low_variance.index, inplace=True, errors='ignore')

# 3. Verify the new shape of the dataset
print(f"Remaining features count: {radiomics_features.shape[1]}")

In [ ]:
# Total count of zeros across the entire dataset
total_zeros = (radiomics_features == 0).sum().sum()
print(f"Total zero values in the dataset: {total_zeros}")

# Detailed breakdown: Count of zeros per feature (only showing those with zeros)
zeros_per_column = (radiomics_features == 0).sum()
features_with_zeros = zeros_per_column[zeros_per_column > 0]

if not features_with_zeros.empty:
    print("\n--- Zero Values Per Feature ---")
    print(features_with_zeros)
else:
    print("\nData Integrity Check Passed: No zero values found.")

In [ ]:
# Correlation Heatmap for the first 20 features
plt.figure(figsize=(12,10))
corr_matrix = radiomics_features.iloc[:, :20].corr()
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title('Feature Correlation Heatmap (Checking for Redundancy)')
plt.show()

In [ ]:
# Check for redundant features
corr_matrix = radiomics_features.iloc[:, :25].corr() # Check a subset for visibility
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap: Identifying Redundant Radiomics Features')
plt.show()

In [ ]:
# Feature/Label Separation
y = radiomics_features['dx']
X = radiomics_features.drop(['image_id', 'dx'], axis=1)

# Scaling: Critical for stable feature importance rankings
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Training/Testing Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Initialize and train the classifier
# Using a depth of 50 to capture complex non-linear relationships
clf = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=42)
clf.fit(X_train, y_train)

# Generate predictions for evaluation
y_pred = clf.predict(X_test)

In [ ]:
# Print high-level metrics
print(f"Classification Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Calculate the variance for every extracted radiomics feature
variances = X.var()

# Sort features from highest variance to lowest
# High variance usually indicates a feature that captures significant differences 
# between different types of lesions.
sorted_variances = variances.sort_values(ascending=False)

# Visualization of the top 125 features
plt.figure(figsize=(12, 6)) # Increased width for better label readability
plt.bar(sorted_variances.index[:125], sorted_variances[:125], color='skyblue')

# Labeling and Formatting
plt.title('Top 125 Features by Variance (Sorted)', fontsize=14)
plt.xlabel('Radiomics Features', fontsize=12)
plt.ylabel('Variance Value', fontsize=12)

# Rotate labels 90 degrees to prevent overlapping
plt.xticks(rotation=90, fontsize=8) 

# Focus the view on a specific range to see the distribution more clearly
plt.ylim([0, 3000])

plt.tight_layout()
# plt.savefig('Feature_Variance_Analysis.png', dpi=300)
plt.show()

In [ ]:
# Initialize components
smt = SMOTE(random_state=42)
clf = RandomForestClassifier(random_state=42)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Select top 125 features based on variance
selected_features = sorted_variances.index[:125]
X_selected = X[selected_features]

acc_score_kfold = []

for fold, (train_ind, test_ind) in enumerate(kf.split(X_selected, y)):
    print(f"--- Processing Fold {fold + 1} ---")
    
    # Split Data
    X_train, X_test = X_selected.iloc[train_ind, :], X_selected.iloc[test_ind, :]
    y_train, y_test = y[train_ind], y[test_ind]
    
    # Handle Class Imbalance
    # SMOTE creates synthetic samples for minority classes in the training set
    x_resampled, y_resampled = smt.fit_resample(X_train, y_train)

    # Feature Scaling
    # Scaler is fit ONLY on training data to prevent data leakage
    sc = StandardScaler()
    X_train_scaled = sc.fit_transform(x_resampled)
    X_test_scaled = sc.transform(X_test)

    # Model Training
    clf.fit(X_train_scaled, y_resampled)

    # Predictions
    pred_values = clf.predict(X_test_scaled)
    y_pred_proba = clf.predict_proba(X_test_scaled)

    # Performance Evaluation
    # Accuracy
    acc = accuracy_score(y_test, pred_values)
    acc_score_kfold.append(acc)
    
    # ROC-AUC (One-vs-Rest)
    roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')
    
    print(f"Fold {fold+1} Accuracy: {acc:.4f}")
    print(f"Fold {fold+1} Multi-class ROC-AUC: {roc_auc:.4f}")
    print(classification_report(y_test, pred_values))

    # Sensitivity & Specificity calculation per class
    res = []
    for l in range(7): # 7 classes in HAM10000
        # Calculate precision, recall (sensitivity), and support
        precision, recall, _, _ = precision_recall_fscore_support(
            np.array(y_test) == l, 
            np.array(pred_values) == l, 
            pos_label=True, 
            average=None
        )
        # recall[0] is Specificity (True Negative Rate), recall[1] is Sensitivity (True Positive Rate)
        res.append([l, recall[1], recall[0]])
    
    print("Class-wise Sensitivity and Specificity:")
    print(pd.DataFrame(res, columns=['class', 'sensitivity', 'specificity']))

    # Visualization
    xp = confusion_matrix(y_test, pred_values)
    plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
    plot_confusion_matrix(xp, plot_labels) # Assumes plot_confusion_matrix is defined
    plt.show()